In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

2.2.0


In [3]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [20]:
train_data, test_data = imdb['train'], imdb['test']
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for sentence, label in train_data:
    training_sentences.append(sentence.numpy().decode('utf8'))
    training_labels.append(label.numpy())

for sentence, label in test_data:
    testing_sentences.append(sentence.numpy().decode('utf8'))
    testing_labels.append(label.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [13]:
vocab_size = 1000
embedding_dim = 16
trunc_type = 'post'
oov_token = '<OOV>'
max_length = 120

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [21]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 27,533
Trainable params: 27,533
Non-trainable params: 0
_________________________________________________________________


In [22]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 2s 3ms/step - loss: 0.5425 - accuracy: 0.7065 - val_loss: 0.3958 - val_accuracy: 0.8174
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.3902 - accuracy: 0.8240 - val_loss: 0.3807 - val_accuracy: 0.8272
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.3426 - accuracy: 0.8506 - val_loss: 0.3936 - val_accuracy: 0.8191
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2883 - accuracy: 0.8839 - val_loss: 0.4363 - val_accuracy: 0.8044
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.2346 - accuracy: 0.9106 - val_loss: 0.4919 - val_accuracy: 0.7957
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1833 - accuracy: 0.9348 - val_loss: 0.5624 - val_accuracy: 0.7876
Epoch 7/10
782/782 [==============================] - 2s 3ms/step - loss: 0.1376 - accuracy: 0.9561 - val_loss: 0.6584 - val_accuracy: 0.7822
Epoch 

In [23]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(1000, 16)
